In [1]:
########## import packages ##########
import pyspark
from pyspark.sql import SQLContext 
from textblob import Word
from nltk.corpus import stopwords
from textblob import TextBlob

In [2]:
########## create sql context ##########
spark_conf = pyspark.SparkConf()
spark_context = pyspark.SparkContext
sc = spark_context.getOrCreate()
sqlContext = SQLContext(sc) 

In [3]:
review_rdd = sqlContext.read.json('yelp/yelp_academic_dataset_review.json').rdd

In [4]:
review_rdd.count()

8635403

In [5]:
review_rdd.take(1)[0]

Row(business_id='buF9druCkbuXLX526sGELQ', cool=1, date='2014-10-11 03:34:02', funny=1, review_id='lWC-xP3rd6obsecCYsGZRg', stars=4.0, text="Apparently Prides Osteria had a rough summer as evidenced by the almost empty dining room at 6:30 on a Friday night. However new blood in the kitchen seems to have revitalized the food from other customers recent visits. Waitstaff was warm but unobtrusive. By 8 pm or so when we left the bar was full and the dining room was much more lively than it had been. Perhaps Beverly residents prefer a later seating. \n\nAfter reading the mixed reviews of late I was a little tentative over our choice but luckily there was nothing to worry about in the food department. We started with the fried dough, burrata and prosciutto which were all lovely. Then although they don't offer half portions of pasta we each ordered the entree size and split them. We chose the tagliatelle bolognese and a four cheese filled pasta in a creamy sauce with bacon, asparagus and grana

In [6]:
review_text = review_rdd.map(lambda r: (r.business_id, r.stars,r.text))

In [7]:
review_text.take(1)

[('buF9druCkbuXLX526sGELQ',
  4.0,
  "Apparently Prides Osteria had a rough summer as evidenced by the almost empty dining room at 6:30 on a Friday night. However new blood in the kitchen seems to have revitalized the food from other customers recent visits. Waitstaff was warm but unobtrusive. By 8 pm or so when we left the bar was full and the dining room was much more lively than it had been. Perhaps Beverly residents prefer a later seating. \n\nAfter reading the mixed reviews of late I was a little tentative over our choice but luckily there was nothing to worry about in the food department. We started with the fried dough, burrata and prosciutto which were all lovely. Then although they don't offer half portions of pasta we each ordered the entree size and split them. We chose the tagliatelle bolognese and a four cheese filled pasta in a creamy sauce with bacon, asparagus and grana frita. Both were very good. We split a secondi which was the special Berkshire pork secreto, which wa

In [8]:
review_text_rdd = sqlContext.createDataFrame(review_text,["business_id","stars","text"]).rdd
review_text_rdd.take(1)

[Row(business_id='buF9druCkbuXLX526sGELQ', stars=4.0, text="Apparently Prides Osteria had a rough summer as evidenced by the almost empty dining room at 6:30 on a Friday night. However new blood in the kitchen seems to have revitalized the food from other customers recent visits. Waitstaff was warm but unobtrusive. By 8 pm or so when we left the bar was full and the dining room was much more lively than it had been. Perhaps Beverly residents prefer a later seating. \n\nAfter reading the mixed reviews of late I was a little tentative over our choice but luckily there was nothing to worry about in the food department. We started with the fried dough, burrata and prosciutto which were all lovely. Then although they don't offer half portions of pasta we each ordered the entree size and split them. We chose the tagliatelle bolognese and a four cheese filled pasta in a creamy sauce with bacon, asparagus and grana frita. Both were very good. We split a secondi which was the special Berkshire 

In [9]:
re=review_text_rdd.map(lambda x:(x.business_id,x.stars,TextBlob(x.text).sentiment[0]))

In [10]:
re.take(1)

[('buF9druCkbuXLX526sGELQ', 4.0, 0.14174312246892892)]

In [11]:
re1=review_text_rdd.map(lambda x:(x.business_id,x.stars,TextBlob(x.text).sentiment[1]))

In [12]:
re1.take(1)

[('buF9druCkbuXLX526sGELQ', 4.0, 0.44589163524647396)]

In [13]:
x = re.join(re1)

In [14]:
x

PythonRDD[31] at RDD at PythonRDD.scala:53

In [ ]:
review_x = sqlContext.createDataFrame(x,["business_id","stars","polarity","subjectivity"]).rdd

In [ ]:
review_x.take(1)

In [15]:
final_review= x.map(lambda r: (r.stars*r.polarity*r.subjectivity))

In [ ]:
final_review.take(1)